***Important notes***:

Before any running please upload WellXplain.csv in the session storage. To do so, you can use the Files section in the left side of this page.

Create the path in the path variable in the first cell. Or you can create your own path for saving results.

You also need API key for access to GPT 4 API which would need payment. You need to insert your key into the appropariate places where it mention openai.api_key="your key."

In the very few cells, we used append() function to append samples extracted from each class to gether. If it won't work you should use concat() function.

# Requirements and Data preparation

In [1]:
path ="/content/drive/MyDrive/ACL_code/results/GPT3.5/" # the path for saving results

In [2]:
from tqdm.autonotebook import tqdm

<ipython-input-2-d05506e76d3e>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import os
import pickle

In [5]:
# reading data
import pandas as pd
data = pd.read_csv('WellXplain.csv',index_col=0)

In [6]:
import numpy as np
def convert_to_one_hot(input_vector):
  input_vector_array = np.array(input_vector)
  input_vector_one_hot = np.zeros((input_vector_array.size, 4))
  input_vector_one_hot[np.arange(input_vector_array.size), input_vector_array-1] = 1
  return input_vector_one_hot

# gpt 3.5 zero shot and few shots

In [7]:
!pip install openai --quiet
# !pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [8]:
import base64
import requests
import re
import openai
from openai import OpenAI
import json


In [9]:
model = "gpt-3.5-turbo"

def generate_message(data, formatted_prompt):
  messages = [{"role": "system", "content": "You are a helpful assistant who can classify wellness dimensions/aspects and provide parts of the input you mostly consider for your decision."}]
  # print(len(data))
  for i in range(len(data)):
    messages.append({"role": "user", "content": data.iloc[i]['Text']})
    messages.append({"role": "assistant", "content": "label:"+str(data.iloc[i]['Aspect'])+"\n explanation:"+data.iloc[i]['Explanations']})
  messages.append({"role": "user", "content": formatted_prompt})
  return messages

def gpt_explanation_prediction(post, model=model):
    # client = OpenAI(api_key=os.environ["sk-your key"])
    client = openai
    openai.api_key ="sk-..." # Please insert your api key for gpt 4 here

    prediction = 0
    explanation=""


    # The "simple" here is intentional: I tried multiple variants: awesome, nice, cool.
    # but starting with simple and refining leads to the lower error rates.
    prompt = f"""
    First, understand the following definitions:
    Physical Aspect (PA): Physical wellness fosters healthy dietary practices while discouraging harmful behaviors like tobacco use, drug misuse, and excessive alcohol consumption. Achieving optimal physical wellness involves regular physical activity, sufficient sleep, vitality, enthusiasm, and beneficial eating habits. Body shaming can negatively affect physical well-being by increasing awareness of medical history and appearance issues.
    Intellectual Aspect (IA): Utilizing intellectual and cultural activities, both inside and outside the classroom, and leveraging human and learning resources enhance the wellness of an individual by nurturing intellectual growth and stimulation.
    Vocational Aspect (VA): The Vocational Dimension acknowledges the role of personal gratification and enrichment derived from one's occupation in shaping life satisfaction. It influences an individual's perspective on creative problem-solving, professional development, and the management of financial obligations.
    Social Aspect (SA): The Social Dimension highlights the interplay between society and the natural environment, increasing individuals' awareness of their role in society and their impact on ecosystems. Social bonds enhance interpersonal traits, enabling a better understanding and appreciation of cultural influences.
    Spiritual Aspect (SpA): The Spiritual Dimension involves seeking the meaning and purpose of human life, appreciating its vastness and natural forces, and achieving harmony within oneself.
    Emotional Aspect (EA): The Emotional Dimension enhances self-awareness and positivity, promoting better emotional control, realistic self-appraisal, independence, and effective stress management.


    Now, you will be given a textual post. Classify the post into one of these labels: 1, 2, 3, or 4.
    If the post is physical aspect, return 1; if it is either intellectual or vocational aspect, or both of these aspects, return 2;
    if the post is social aspect, return 3; and if the post is either spiritual or emotional, or both of these aspect, return 4.
    After that, just list (in order based on the importance) at most four parts of the post were the most important parts you considered to make your decision.
    You should provide your output as a python list with two values the first one represents your prediction (1, 2, 3, or 4) and
    the second one represents the parts you consider the most important parts used for your prediction, like the following:
    [value1, value2]

    Textual post: {post}


    """
    formatted_prompt = prompt.format(post=post)

    messages = generate_message(train, formatted_prompt)
    # print(messages)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who can classify wellness aspects and provide parts of the post that you consider to make your prediction based on them."},
            {"role": "user", "content": formatted_prompt},
        ]
    )
    return response

def extract_explanation_from_response(response):
    # Check if the response is a dictionary and has the 'choices' key
    if isinstance(response, openai.types.chat.chat_completion.ChatCompletion):
      content = response.choices[0].message.content
    elif isinstance(response, dict) and 'choices' in response:
      content = response['choices'][0]['message']['content']
    elif isinstance(response, str) and isinstance(json.loads(response), dict) and 'choices' in json.loads(response):
      content = json.loads(response)['choices'][0]['message']['content']
    else:
      print("Invalid response format.")
      return None
    extracted_response=content.split("\n")
    return extracted_response[0], extracted_response[1]


In [10]:
def few_shots_preparing(number_prompt_sample):
  label1 = data[data["Aspect"]==1].sample(n=number_prompt_sample, replace=False, random_state = random_state)
  label2 = data[data["Aspect"]==2].sample(n=number_prompt_sample, replace=False, random_state = random_state)
  label3 = data[data["Aspect"]==3].sample(n=number_prompt_sample, replace=False, random_state = random_state)
  label4 = data[data["Aspect"]==4].sample(n=number_prompt_sample, replace=False, random_state = random_state)

  data_balanced = label1
  data_balanced = data_balanced._append(label2)
  data_balanced = data_balanced._append(label3)
  data_balanced = data_balanced._append(label4)
  # data_balanced.columns=["index","Text","Aspect","Explanations"]

  return data_balanced

In [19]:
def validation_data(number_validation_sample, train):
  data1 = data.drop(train.index)
  return data1


In [11]:
def main(file_name,val_data):
  results = []
  if os.path.isfile(file_name):
    with open(file_name, 'rb') as f:
      results = pickle.load(f)

  for i in tqdm(range(len(val_data))):

    output = gpt_explanation_prediction(val_data.iloc[i]['Text'])
    content = output.choices[0].message.content

    results.append([val_data.iloc[i]['tmp'], val_data.iloc[i]['Text'],val_data.iloc[i]['Aspect'],val_data.iloc[i]['Explanations'], content])
    if i%30 ==0:
      with open(file_name, 'wb') as f:
        pickle.dump(results, f)
  with open(file_name, 'wb') as f:
        pickle.dump(results, f)

  return results


In [12]:
def retreive_results(file_name):
  with open(file_name, 'rb') as f:
      results = pickle.load(f)
  manual_checking = []
  true_label = []
  prediction= []
  # for i in range(len(results)):
  for itr, item in enumerate(results):
    try:
      output = eval(item[4])
      true_label.append(item[2])
      prediction.append(output[0])
    except:
      manual_checking.append(itr)
      continue
  return true_label, prediction

In [13]:
pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.9 MB/s eta 0:00:00


In [14]:
import tensorflow_addons as tfa
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
def metrics():
  target_List =["Aspect1","Aspect2","Aspect3","Aspect4"]
  print(classification_report(true_label, prediction, labels =np.array(range(1, len(target_List)+1)),target_names=target_List, digits=4,zero_division=0))

  metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=4)
  metric.update_state(true_label_one_hot,prediction_one_hot)
  result = metric.result()
  print(result.numpy())
  print('target_List:', target_List)

  val = pd.DataFrame(true_label_one_hot, columns = target_List)
  fin = pd.DataFrame(prediction_one_hot, columns = target_List)

  from sklearn.metrics import matthews_corrcoef
  print(matthews_corrcoef(val["Aspect1"],fin["Aspect1"]))

  from sklearn.metrics import matthews_corrcoef
  print(matthews_corrcoef(val["Aspect2"],fin["Aspect2"]))

  from sklearn.metrics import matthews_corrcoef
  print(matthews_corrcoef(val["Aspect3"],fin["Aspect3"]))

  from sklearn.metrics import matthews_corrcoef
  print(matthews_corrcoef(val["Aspect4"],fin["Aspect4"]))

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [15]:
def calculate_acc(results):
  c = 0
  manual_checking = []
  # for i in range(len(results)):
  for iter, item in enumerate(results):
    i = item[0]
    try:
      output = eval(item[4])
    except:
      manual_checking.append(iter)
      continue
    # print(item[2], output[0])
    if item[2]==output[0]:
      c+=1
  return c/(len(results)-len(manual_checking))

In [16]:
random_state = 42

In [ ]:
# Few-Shots, 5 shots per class

number_prompt_sample = 5 # number of shots for each class
number_validation_sample = 100 # number of test samples
file_name =path+"gpt35_few_shots_5.pkl"

# reading data
import pandas as pd
data = pd.read_csv('WellXplain.csv',index_col=0)

train =few_shots_preparing(number_prompt_sample)
val_data = validation_data(number_validation_sample, train)
val_data['tmp'] =val_data.index

results = main(file_name = file_name, val_data=val_data)
true_label, prediction = retreive_results(file_name)
true_label_one_hot = convert_to_one_hot(true_label)
prediction_one_hot = convert_to_one_hot(prediction)

metrics()
print("Accuracy:", calculate_acc(results))

In [ ]:
# Few-Shots, 10 shots per class

number_prompt_sample = 10 # number of shots for each class
number_validation_sample = 100 # number of test samples
file_name =path+"gpt35_few_shots_10.pkl"

# reading data
import pandas as pd
data = pd.read_csv('WellXplain.csv',index_col=0)

train =few_shots_preparing(number_prompt_sample)
val_data = validation_data(number_validation_sample, train)
val_data['tmp'] =val_data.index

results = main(file_name = file_name, val_data=val_data)
true_label, prediction = retreive_results(file_name)
true_label_one_hot = convert_to_one_hot(true_label)
prediction_one_hot = convert_to_one_hot(prediction)

metrics()
print("Accuracy:", calculate_acc(results))

In [ ]:
# Zero-Shots, 0 shots per class

number_prompt_sample = 0 # number of shots for each class
number_validation_sample = 100 # number of test samples
file_name =path+"gpt35_zero_shots.pkl"

# reading data
import pandas as pd
data = pd.read_csv('WellXplain.csv',index_col=0)

train =few_shots_preparing(number_prompt_sample)
val_data = validation_data(number_validation_sample, train)
val_data['tmp'] =val_data.index

results = main(file_name = file_name, val_data=val_data)
true_label, prediction = retreive_results(file_name)
true_label_one_hot = convert_to_one_hot(true_label)
prediction_one_hot = convert_to_one_hot(prediction)

metrics()
print("Accuracy:", calculate_acc(results))

  0%|          | 0/3092 [00:00<?, ?it/s]

              precision    recall  f1-score   support

     Aspect1     0.8145    0.2049    0.3274       493
     Aspect2     0.8402    0.4256    0.5650       383
     Aspect3     0.8369    0.1667    0.2780       708
     Aspect4     0.2467    0.9424    0.3911       399

    accuracy                         0.3822      1983
   macro avg     0.6846    0.4349    0.3904      1983
weighted avg     0.7132    0.3822    0.3684      1983

0.31349567
target_List: ['Aspect1', 'Aspect2', 'Aspect3', 'Aspect4']
0.3381576372798971
0.539767325912764
0.27708714731650147
0.20684303228693196
Accuracy: 0.3822491174987393


<string>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<string>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
